In [10]:
# Crea un netcdf con el viento a 925 hPa alineado con respecto
# a la fecha del onset de la temporada de lluvias.

import os

import rioxarray

import pandas as pd
import numpy as np

import xarray as xr

In [11]:
# Datos.

path_d = "../data/ERA5/"
path_r = "../results/onset/"

# Variable, nivel de presión y región.
vars = [ "u", "v", "sst", "olr" ]
levels = [ "925" ]
region = [ "mexico" ]

# Viento zonal.
fname = region[0] + "_" + vars[0] + "_" + levels[0] + "_daily.grib"
U = xr.open_dataset( path_d + fname, engine = "cfgrib" )

# Viento meridional.
fname = region[0] + "_" + vars[1] + "_" + levels[0] + "_daily.grib"
V = xr.open_dataset( path_d + fname, engine = "cfgrib" )

# Unimos en un solo netcdf.
ds = xr.merge( ( U, V ) )

ds

<xarray.Dataset>
Dimensions:        (time: 14965, latitude: 161, longitude: 281)
Coordinates:
  * time           (time) datetime64[ns] 1981-01-01T06:00:00 ... 2021-12-31T0...
    step           timedelta64[ns] 00:00:00
    isobaricInhPa  int64 925
  * latitude       (latitude) float64 40.0 39.75 39.5 39.25 ... 0.5 0.25 0.0
  * longitude      (longitude) float64 -120.0 -119.8 -119.5 ... -50.25 -50.0
    valid_time     (time) datetime64[ns] 1981-01-01T06:00:00 ... 2021-12-31T0...
Data variables:
    u              (time, latitude, longitude) float32 ...
    v              (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-01-28T11:13:27 GRIB to CDM+CF via cfgrib-0....

In [12]:
# Composite de la distribución espacial.

# Años comprendidos en CHIRPS.
years = list( range(1981, 2021) )

# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos alas fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)

# Cargamos la información de desfase para cada año.
shift = pd.read_csv(path_r + "shift_CHIRPS.csv",
    index_col = "Año", squeeze = True)
# Fecha pivote del desfase. 
delta = shift.max()
shift = shift.max() - shift

# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift.iloc[i] } )

# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)

# Guardamos el netcdf.
composite.to_netcdf(path_r + "onset_composite_U_" + levels[0] + "_mean.nc")

composite

<xarray.Dataset>
Dimensions:        (latitude: 161, longitude: 281, dayofyear: 121)
Coordinates:
    step           timedelta64[ns] 00:00:00
    isobaricInhPa  int64 925
  * latitude       (latitude) float64 40.0 39.75 39.5 39.25 ... 0.5 0.25 0.0
  * longitude      (longitude) float64 -120.0 -119.8 -119.5 ... -50.25 -50.0
  * dayofyear      (dayofyear) int64 -60 -59 -58 -57 -56 -55 ... 56 57 58 59 60
    spatial_ref    int64 0
Data variables:
    u              (dayofyear, latitude, longitude) float32 1.252 ... -8.027
    v              (dayofyear, latitude, longitude) float32 0.822 ... -0.1482

In [13]:
# Composite de la distribución espacial, umbral de 3 mm/día.

# Años comprendidos en CHIRPS.
years = list( range(1981, 2021) )

# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos alas fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)

# Cargamos la información de desfase para cada año.
shift = pd.read_csv(path_r + "shift_CHIRPS_3.csv",
    index_col = "Año", squeeze = True)
# Fecha pivote del desfase. 
delta = shift.max()
shift = shift.max() - shift

# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift.iloc[i] } )

# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)

# Guardamos el netcdf.
composite.to_netcdf(path_r + "onset_composite_U_" + levels[0] + "_mean_3.nc")

composite

<xarray.Dataset>
Dimensions:        (latitude: 161, longitude: 281, dayofyear: 121)
Coordinates:
    step           timedelta64[ns] 00:00:00
    isobaricInhPa  int64 925
  * latitude       (latitude) float64 40.0 39.75 39.5 39.25 ... 0.5 0.25 0.0
  * longitude      (longitude) float64 -120.0 -119.8 -119.5 ... -50.25 -50.0
  * dayofyear      (dayofyear) int64 -60 -59 -58 -57 -56 -55 ... 56 57 58 59 60
    spatial_ref    int64 0
Data variables:
    u              (dayofyear, latitude, longitude) float32 1.326 ... -7.969
    v              (dayofyear, latitude, longitude) float32 0.03336 ... -0.5712

In [14]:
# Temperatura superficial del océano.
fname = region[0] + "_" + vars[2] + "_daily.grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )

ds

<xarray.Dataset>
Dimensions:     (time: 14965, latitude: 221, longitude: 281)
Coordinates:
  * time        (time) datetime64[ns] 1981-01-01T11:30:00 ... 2021-12-31T11:3...
    step        timedelta64[ns] ...
    surface     int64 ...
  * latitude    (latitude) float64 50.0 49.75 49.5 49.25 ... -4.5 -4.75 -5.0
  * longitude   (longitude) float64 -135.0 -134.8 -134.5 ... -65.5 -65.25 -65.0
    valid_time  (time) datetime64[ns] ...
Data variables:
    sst         (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-01-28T11:27:19 GRIB to CDM+CF via cfgrib-0....

In [15]:
# Temperatura superficial del océano.
# Composite de la distribución espacial, variable.

# Años comprendidos en CHIRPS.
years = list( range(1981, 2021) )

# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos alas fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)

# Cargamos la información de desfase para cada año.
shift = pd.read_csv(path_r + "shift_CHIRPS.csv",
    index_col = "Año", squeeze = True)
# Fecha pivote del desfase. 
delta = shift.max()
shift = shift.max() - shift

# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift.iloc[i] } )

# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)

# Guardamos el netcdf.
composite.to_netcdf(path_r + "onset_composite_sst_mean.nc")

composite

<xarray.Dataset>
Dimensions:      (latitude: 221, longitude: 281, dayofyear: 121)
Coordinates:
    step         timedelta64[ns] 00:00:00
    surface      int64 0
  * latitude     (latitude) float64 50.0 49.75 49.5 49.25 ... -4.5 -4.75 -5.0
  * longitude    (longitude) float64 -135.0 -134.8 -134.5 ... -65.5 -65.25 -65.0
  * dayofyear    (dayofyear) int64 -60 -59 -58 -57 -56 -55 ... 55 56 57 58 59 60
    spatial_ref  int64 0
Data variables:
    sst          (dayofyear, latitude, longitude) float32 280.2 280.3 ... nan

In [16]:
# Temperatura superficial del océano.
fname = region[0] + "_" + vars[3] + "_daily.grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )

ds

<xarray.Dataset>
Dimensions:     (time: 14965, latitude: 161, longitude: 281)
Coordinates:
  * time        (time) datetime64[ns] 1981-01-01T06:00:00 ... 2021-12-31T06:0...
    step        timedelta64[ns] ...
    surface     int64 ...
  * latitude    (latitude) float64 40.0 39.75 39.5 39.25 ... 0.75 0.5 0.25 0.0
  * longitude   (longitude) float64 -120.0 -119.8 -119.5 ... -50.5 -50.25 -50.0
    valid_time  (time) datetime64[ns] ...
Data variables:
    ttr         (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-01-28T11:30:58 GRIB to CDM+CF via cfgrib-0....

In [17]:
# Temperatura superficial del océano.
# Composite de la distribución espacial, variable.

# Años comprendidos en CHIRPS.
years = list( range(1981, 2021) )

# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos alas fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)

# Cargamos la información de desfase para cada año.
shift = pd.read_csv(path_r + "shift_CHIRPS.csv",
    index_col = "Año", squeeze = True)
# Fecha pivote del desfase. 
delta = shift.max()
shift = shift.max() - shift

# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift.iloc[i] } )

# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)

# Guardamos el netcdf.
composite.to_netcdf(path_r + "onset_composite_olr_mean.nc")

composite

<xarray.Dataset>
Dimensions:      (latitude: 161, longitude: 281, dayofyear: 121)
Coordinates:
    step         timedelta64[ns] 00:00:00
    surface      int64 0
  * latitude     (latitude) float64 40.0 39.75 39.5 39.25 ... 0.75 0.5 0.25 0.0
  * longitude    (longitude) float64 -120.0 -119.8 -119.5 ... -50.5 -50.25 -50.0
  * dayofyear    (dayofyear) int64 -60 -59 -58 -57 -56 -55 ... 55 56 57 58 59 60
    spatial_ref  int64 0
Data variables:
    ttr          (dayofyear, latitude, longitude) float32 -7.786e+05 ... -9.5...